In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import time
import re
import os

%config InlineBackend.figure_format = 'png'
plt.rcParams['figure.dpi']= 300

In [3]:
df_raw = pd.read_csv('baseballdatabank-master/core/teams.csv')

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2865 entries, 0 to 2864
Data columns (total 48 columns):
yearID            2865 non-null int64
lgID              2815 non-null object
teamID            2865 non-null object
franchID          2865 non-null object
divID             1348 non-null object
Rank              2865 non-null int64
G                 2865 non-null int64
Ghome             2466 non-null float64
W                 2865 non-null int64
L                 2865 non-null int64
DivWin            1320 non-null object
WCWin             684 non-null object
LgWin             2837 non-null object
WSWin             2508 non-null object
R                 2865 non-null int64
AB                2865 non-null int64
H                 2865 non-null int64
2B                2865 non-null int64
3B                2865 non-null int64
HR                2865 non-null int64
BB                2864 non-null float64
SO                2745 non-null float64
SB                2739 non-null float64
CS  

In [10]:
df_modern = df_raw[df_raw.yearID > 2001]

In [11]:
df_modern.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
2385,2002,AL,ANA,ANA,W,2,162,81.0,99,63,...,151,0.986,Anaheim Angels,Edison International Field,2305547.0,100,99,ANA,ANA,ANA
2386,2002,NL,ARI,ARI,W,1,162,81.0,98,64,...,116,0.985,Arizona Diamondbacks,Bank One Ballpark,3198977.0,111,111,ARI,ARI,ARI
2387,2002,NL,ATL,ATL,E,1,161,81.0,101,59,...,170,0.982,Atlanta Braves,Turner Field,2603484.0,102,101,ATL,ATL,ATL
2388,2002,AL,BAL,BAL,E,4,162,81.0,67,95,...,173,0.985,Baltimore Orioles,Oriole Park at Camden Yards,2682439.0,95,96,BAL,BAL,BAL
2389,2002,AL,BOS,BOS,E,2,162,81.0,93,69,...,140,0.983,Boston Red Sox,Fenway Park II,2650862.0,103,102,BOS,BOS,BOS


In [13]:
df_modAL = df_modern[df_modern.lgID == 'AL']

In [15]:
df_modAL.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
2385,2002,AL,ANA,ANA,W,2,162,81.0,99,63,...,151,0.986,Anaheim Angels,Edison International Field,2305547.0,100,99,ANA,ANA,ANA
2388,2002,AL,BAL,BAL,E,4,162,81.0,67,95,...,173,0.985,Baltimore Orioles,Oriole Park at Camden Yards,2682439.0,95,96,BAL,BAL,BAL
2389,2002,AL,BOS,BOS,E,2,162,81.0,93,69,...,140,0.983,Boston Red Sox,Fenway Park II,2650862.0,103,102,BOS,BOS,BOS
2390,2002,AL,CHA,CHW,C,2,162,81.0,81,81,...,157,0.984,Chicago White Sox,Comiskey Park II,1676911.0,102,101,CHW,CHA,CHA
2393,2002,AL,CLE,CLE,C,3,162,81.0,74,88,...,161,0.981,Cleveland Indians,Jacobs Field,2616940.0,97,98,CLE,CLE,CLE


In [17]:
df_modAL.info() # expecting 229 rows - 14 teams x 16 years plus last 5 yrs of HOU after league switch

<class 'pandas.core.frame.DataFrame'>
Int64Index: 229 entries, 2385 to 2863
Data columns (total 48 columns):
yearID            229 non-null int64
lgID              229 non-null object
teamID            229 non-null object
franchID          229 non-null object
divID             229 non-null object
Rank              229 non-null int64
G                 229 non-null int64
Ghome             229 non-null float64
W                 229 non-null int64
L                 229 non-null int64
DivWin            229 non-null object
WCWin             229 non-null object
LgWin             229 non-null object
WSWin             229 non-null object
R                 229 non-null int64
AB                229 non-null int64
H                 229 non-null int64
2B                229 non-null int64
3B                229 non-null int64
HR                229 non-null int64
BB                229 non-null float64
SO                229 non-null float64
SB                229 non-null float64
CS                229 no

In [18]:
df_modAL['G'].describe() # concerned that a few seasons have one game too few or too many; should be 162

count    229.000000
mean     161.982533
std        0.295657
min      161.000000
25%      162.000000
50%      162.000000
75%      162.000000
max      163.000000
Name: G, dtype: float64

In [24]:
df_modAL['yrtm'] = df_modAL['yearID'].apply(str) + df_modAL['franchID']

/Users/kellyjones/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_modAL.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro,yrtm
2385,2002,AL,ANA,ANA,W,2,162,81.0,99,63,...,0.986,Anaheim Angels,Edison International Field,2305547.0,100,99,ANA,ANA,ANA,2002ANA
2388,2002,AL,BAL,BAL,E,4,162,81.0,67,95,...,0.985,Baltimore Orioles,Oriole Park at Camden Yards,2682439.0,95,96,BAL,BAL,BAL,2002BAL
2389,2002,AL,BOS,BOS,E,2,162,81.0,93,69,...,0.983,Boston Red Sox,Fenway Park II,2650862.0,103,102,BOS,BOS,BOS,2002BOS
2390,2002,AL,CHA,CHW,C,2,162,81.0,81,81,...,0.984,Chicago White Sox,Comiskey Park II,1676911.0,102,101,CHW,CHA,CHA,2002CHW
2393,2002,AL,CLE,CLE,C,3,162,81.0,74,88,...,0.981,Cleveland Indians,Jacobs Field,2616940.0,97,98,CLE,CLE,CLE,2002CLE


In [27]:
df_modAL.set_index('yrtm').reset_index()

,yrtm,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,2002ANA,2002,AL,ANA,ANA,W,2,162,81.0,99,...,151,0.986,Anaheim Angels,Edison International Field,2305547.0,100,99,ANA,ANA,ANA
1,2002BAL,2002,AL,BAL,BAL,E,4,162,81.0,67,...,173,0.985,Baltimore Orioles,Oriole Park at Camden Yards,2682439.0,95,96,BAL,BAL,BAL
2,2002BOS,2002,AL,BOS,BOS,E,2,162,81.0,93,...,140,0.983,Boston Red Sox,Fenway Park II,2650862.0,103,102,BOS,BOS,BOS
3,2002CHW,2002,AL,CHA,CHW,C,2,162,81.0,81,...,157,0.984,Chicago White Sox,Comiskey Park II,1676911.0,102,101,CHW,CHA,CHA
4,2002CLE,2002,AL,CLE,CLE,C,3,162,81.0,74,...,161,0.981,Cleveland Indians,Jacobs Field,2616940.0,97,98,CLE,CLE,CLE
5,2002DET,2002,AL,DET,DET,C,5,161,80.0,55,...,148,0.977,Detroit Tigers,Comerica Park,1503623.0,92,95,DET,DET,DET
6,2002KCR,2002,AL,KCA,KCR,C,4,162,77.0,62,...,153,0.979,Kansas City Royals,Kauffman Stadium,1323036.0,110,111,KCR,KCA,KCA
7,2002MIN,2002,AL,MIN,MIN,C,1,161,81.0,94,...,124,0.987,Minnesota Twins,Hubert H Humphrey Metrodome,1924473.0,100,100,MIN,MIN,MIN
8,2002NYY,2002,AL,NYA,NYY,E,1,161,80.0,103,...,117,0.979,New York Yankees,Yankee Stadium II,3465807.0,100,99,NYY,NYA,NYA
9,2002OAK,2002,AL,OAK,OAK,W,1,162,81.0,103,...,144,0.984,Oakland Athletics,Oakland Coliseum,2169811.0,98,97,OAK,OAK,OAK


In [28]:
df_modAL.columns

Index(['yearID', 'lgID', 'teamID', 'franchID', 'divID', 'Rank', 'G', 'Ghome',
       'W', 'L', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA',
       'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP',
       'name', 'park', 'attendance', 'BPF', 'PPF', 'teamIDBR',
       'teamIDlahman45', 'teamIDretro', 'yrtm'],
      dtype='object')

In [ ]:
df_red = df_modAL.loc[:, ['yrtm', 'yearID', 'franchID', 'Rank', 'G', 'W', 'L', 'R', 'RA', 'ERA', '', 'lgID', 'teamID', ]]